2D Pressure Vessel {#2d_pressure_vessel_example}
==================

This workshop demonstrates how to create a basic pressure vessel and
apply a pressure to it.

Objective
---------

In this example we will perform stress analysis of pipe due to internal
pressure. Due to the symmetry in geometry and loading, the strain along
its axis is negligible and therefore we model this system as 2D plane
strain.

Procedure
---------

-   Launch MAPDL instance
-   Converting MAPDL script file to pyMAPDL
    1. Using `pymapdl convert` command utility
    2. Using `pymapdl.convert_script()` method
-   Create function to run the pymapdl script for varying mesh sizes
-   Automate mesh convergence study
-   Plot results of interest

Additional Packages Used
------------------------

-   [Matplotlib](https://matplotlib.org) is used for plotting purposes.
-   [NumPy](https://numpy.org) is used for using NumPy arrays.

Workshop Starts
--------------

After converting the script proceed to modify below where ` your script goes here `


In [ ]:
import matplotlib.pyplot as plt

Launch MAPDL
============


In [ ]:
import numpy as np

from ansys.mapdl.core import launch_mapdl

# start mapdl
mapdl = launch_mapdl()###############your script goes here###########################

Setup the pipe cross section using Python function

We use a function here so we can rebuild the pipe using parameters
rather than calling a script several times.


In [ ]:
def pipe_plane_strain(e, nu, inner_radius, outer_radius, pressure, element_size):
    """Create 2D cross section modeling a pipe."""

    ###############your script goes here###########################

    mapdl.clear()

    mapdl.prep7()


    # Execute below lines within a function to execute in a loop
    mapdl.et(1, 182, "", "", 2)

    mapdl.pcirc(inner_radius, outer_radius, 0, 90)
    mapdl.cm("pipe_profile", "area")
    mapdl.mp("ex", 1, e)
    mapdl.mp("prxy", 1, nu)


    mapdl.aesize("all", element_size)
    mapdl.mshape(0, "2D")
    mapdl.mshkey(1)
    mapdl.cmsel("s", "pipe_profile")
    mapdl.amesh("all")


    mapdl.nsel("s", "loc", "x")
    mapdl.cm("x_fixed", "nodes")
    mapdl.nsel("s", "loc", "y")
    mapdl.cm("y_fixed", "node")

    mapdl.lsel("s", "radius", "", inner_radius)
    mapdl.cm("pressure_edge", "line")
    mapdl.allsel("all", "all")

    mapdl.slashsolu()
    mapdl.antype("static")
    mapdl.d("x_fixed", "ux")
    mapdl.d("y_fixed", "uy")

    mapdl.csys(1)
    mapdl.sfl("pressure_edge", "pres", pressure)
    mapdl.allsel("all", "all")
    mapdl.solve()
    mapdl.finish()

    mapdl.post1()
    mapdl.set(1, 1)


    max_eqv_stress = np.max(mapdl.post_processing.nodal_eqv_stress())
    all_dof = mapdl.mesh.nnum_all
    num_dof = all_dof.size

    return num_dof, max_eqv_stress

Perform the mesh convergence study
==================================


In [ ]:
# Define model input parameters
rad1 = 175  # Internal radius
rad2 = 200  # External radius
pressure = 100

e = 2e5  # Young's modulus
nu = 0.3  # Poisson's ratio

# Define mesh convergence parameters
num_dof = []
max_stress = []

# element size: use log space since mesh converges logarithmically
esizes = np.logspace(1.4, 0, 20)

# run the mesh convergence and output the results on the fly
for esize in esizes:
    dof, eqv_stress = pipe_plane_strain(e, nu, rad1, rad2, pressure, esize)
    num_dof.append(dof)
    max_stress.append(eqv_stress)
    print(f"DOF: {dof:5d}   Stress: {eqv_stress:.2f} MPa")

Plot mesh convergence results
=============================

Draw a dotted line showing the convergence value


In [ ]:
plt.plot(num_dof, max_stress, "b-o")
plt.plot([num_dof[0], num_dof[-1]], [max_stress[-1], max_stress[-1]], "r:")
plt.title("Mesh Convergence Study")
plt.xlabel("Number of DOF")
plt.ylabel("Maximum eqv. Stress (MPa)")
plt.show()

Resume results from last analysis from mesh convergence study


In [ ]:
# Plot the final mesh used
mapdl.allsel("ALL")
mapdl.eplot(
    title="Element Plot",
    line_width=1,
    show_bounds=True,
    cpos="xy",
)

Plot nodal displacement
=======================

Enter post-processing (/POST1) and select the first load step


In [ ]:
mapdl.post1()
mapdl.set(1, 1)

mapdl.post_processing.plot_nodal_displacement(
    "NORM",
    cpos="xy",
    cmap="magma",
)

Plot nodal equivalent stress
============================


In [ ]:
mapdl.post_processing.plot_nodal_eqv_stress(cpos="xy", cmap="magma")

Stop mapdl
==========


In [ ]:
mapdl.exit()